In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
df = pd.read_excel('./run4_correctcarboncosts/AllCarbonNumbers_080922.xlsx')
num_rows = df.shape[0]
df.insert(0, 'Net_Carbon', [0] * num_rows)

START_YEAR = 2021
FOR_TYPES = list(set(df['StandID']))
YEARS = list(set(df['Year']))
MNGS = list(set(df['MgmtID']))

df.sort_values(['StandID', 'MgmtID', 'Year'])


,Net_Carbon,Variable,StandID,MgmtID,Year,Total_Stand_Carbon,Merch_Carbon_Stored
0,0,505_2021_ASV,505,ASV,2021,55.202126,0.000083
4,0,505_2025_ASV,505,ASV,2025,59.408627,0.179192
8,0,505_2030_ASV,505,ASV,2030,63.405056,0.372516
12,0,505_2050_ASV,505,ASV,2050,94.440681,1.162175
1,0,505_2021_IFM,505,IFM,2021,52.616356,1.492325
5,0,505_2025_IFM,505,IFM,2025,54.112751,1.235478
9,0,505_2030_IFM,505,IFM,2030,57.430019,1.043617
13,0,505_2050_IFM,505,IFM,2050,78.380608,0.803028
2,0,505_2021_PLSQ,505,PLSQ,2021,55.080078,0.000000
6,0,505_2025_PLSQ,505,PLSQ,2025,59.402340,0.000000


## What We Want To Do
 - 1) Turn stand carbon into net sequestration
 - 2) Add harvested product
 - 3) Subtract carbon costs

## 1 - Net Sequestration
We add a new column, net_carbon, which for each entry is equal to the net change in Total_Stand_Carbon from
one year to the next.

For 2021, we compare against the no management (PLSQ) for each forest type.

In [3]:
# Baseline extraction - we generate a dataframe which gives the Start Year PLSQ for each forest type
df_baseline = df[df['Year'] == START_YEAR]
df_baseline = df_baseline[df_baseline['MgmtID'] == 'PLSQ']

# We only care about these two columns
df_baseline = df_baseline[['Year', 'StandID', 'Total_Stand_Carbon']]
df_baseline = df_baseline.rename(columns={
    'Total_Stand_Carbon': 'Baseline_Carbon_2021'
})
df_baseline

,Year,StandID,Baseline_Carbon_2021
2,2021,505,55.080078
17,2021,608,45.944687
27,2021,999,7.028458
44,2021,167N,35.021587
52,2021,167S,37.369804


In [5]:
# Calculate Net Carbon
# Step 1: Take difference between year to year

# This is achieved by sorting correctly, and then subtracting consecutive rows
# See: https://datagy.io/pandas-shift/
df_seq = df.sort_values(['StandID', 'MgmtID'])
df_seq['Shifted Carbon'] = df_seq['Total_Stand_Carbon'].shift(periods=1, fill_value=0)
df_seq['Net_Carbon'] = df_seq['Total_Stand_Carbon'] - df_seq['Shifted Carbon']
df_seq

,Net_Carbon,Variable,StandID,MgmtID,Year,Total_Stand_Carbon,Merch_Carbon_Stored,Shifted Carbon
0,55.202126,505_2021_ASV,505,ASV,2021,55.202126,0.000083,0.000000
4,4.206501,505_2025_ASV,505,ASV,2025,59.408627,0.179192,55.202126
8,3.996429,505_2030_ASV,505,ASV,2030,63.405056,0.372516,59.408627
12,31.035625,505_2050_ASV,505,ASV,2050,94.440681,1.162175,63.405056
1,-41.824326,505_2021_IFM,505,IFM,2021,52.616356,1.492325,94.440681
5,1.496395,505_2025_IFM,505,IFM,2025,54.112751,1.235478,52.616356
9,3.317268,505_2030_IFM,505,IFM,2030,57.430019,1.043617,54.112751
13,20.950588,505_2050_IFM,505,IFM,2050,78.380608,0.803028,57.430019
2,-23.300529,505_2021_PLSQ,505,PLSQ,2021,55.080078,0.000000,78.380608
6,4.322262,505_2025_PLSQ,505,PLSQ,2025,59.402340,0.000000,55.080078


In [6]:
# Calculate Net Carbon
# Step 2: For 2021, subtract against PLSQ

# This adds a column with the 2021 PLSQ baseline numbers
df_seq = df_seq.merge(df_baseline, 
         how='left', 
         on=['Year', 'StandID'], 
         validate='many_to_one'
        )

df_seq['First_Year_Net_Carbon'] = df_seq['Total_Stand_Carbon'] - df_seq['Baseline_Carbon_2021']
df_seq['Net_Carbon'] = df_seq['First_Year_Net_Carbon'].fillna(df_seq['Net_Carbon'])
df_seq

,Net_Carbon,Variable,StandID,MgmtID,Year,Total_Stand_Carbon,Merch_Carbon_Stored,Shifted Carbon,Baseline_Carbon_2021,First_Year_Net_Carbon
0,0.122047,505_2021_ASV,505,ASV,2021,55.202126,0.000083,0.000000,55.080078,0.122047
1,4.206501,505_2025_ASV,505,ASV,2025,59.408627,0.179192,55.202126,NaN,NaN
2,3.996429,505_2030_ASV,505,ASV,2030,63.405056,0.372516,59.408627,NaN,NaN
3,31.035625,505_2050_ASV,505,ASV,2050,94.440681,1.162175,63.405056,NaN,NaN
4,-2.463722,505_2021_IFM,505,IFM,2021,52.616356,1.492325,94.440681,55.080078,-2.463722
5,1.496395,505_2025_IFM,505,IFM,2025,54.112751,1.235478,52.616356,NaN,NaN
6,3.317268,505_2030_IFM,505,IFM,2030,57.430019,1.043617,54.112751,NaN,NaN
7,20.950588,505_2050_IFM,505,IFM,2050,78.380608,0.803028,57.430019,NaN,NaN
8,0.000000,505_2021_PLSQ,505,PLSQ,2021,55.080078,0.000000,78.380608,55.080078,0.000000
9,4.322262,505_2025_PLSQ,505,PLSQ,2025,59.402340,0.000000,55.080078,NaN,NaN


In [7]:
# Cleanup - remove now unnecessary rows
df_seq = df_seq.drop(columns=[
    'Shifted Carbon', 
    'Baseline_Carbon_2021', 
    'First_Year_Net_Carbon',
    'Total_Stand_Carbon'
])

In [8]:
# df_mgcost.to_csv('./ObjectiveFile_MngCost_0802022.csv',
#               columns=['Variable', 'Net_Carbon'],
#               index=False
#              )

df_seq.to_csv('./run4_correctcarboncosts/Objective_NetSeq_NoHarvest_08172022.csv',
             columns=['Variable', 'Net_Carbon'],
             index=False
             )

## 2 - Add Carbon Harvest
The Merch_Carbon_Stored column tells us how much carbon total has been stored.
Some managements require management at the start, while some require continual managing.

Cost Frequency:
 - ASV: Every Cycle
 - IFM: Once
 - THNB: Once
 - AWR: Once
 - CAR: Once
 - NAR: Once
 - SAR: Once

For a management that gets managed once, we add the final carbon stored to the 2021 figure.
For ASV, we do it like Net Carbon, taking the new harvest carbon each year.

It's not quite right to do ASV the same way, butttttt I will because I don't see how else to extract
carbon numbers. Doing per cycle ignores the impact of carbon decay :/

In [10]:
once_costs = ['IFM', 'THNB', 'AWR', 'CAR', 'NAR', 'SAR']
repeated_costs = ['ASV']

In [25]:
df_hrv = copy.deepcopy(df_seq)
df_hrv.sort_values(['MgmtID', 'StandID', 'Year'])

,Net_Carbon,Variable,StandID,MgmtID,Year,Merch_Carbon_Stored
0,0.122047,505_2021_ASV,505,ASV,2021,0.000083
1,4.206501,505_2025_ASV,505,ASV,2025,0.179192
2,3.996429,505_2030_ASV,505,ASV,2030,0.372516
3,31.035625,505_2050_ASV,505,ASV,2050,1.162175
16,-18.667194,608_2021_AWR,608,AWR,2021,11.303518
17,-5.682604,608_2025_AWR,608,AWR,2025,9.906696
18,-0.463524,608_2030_AWR,608,AWR,2030,8.775704
19,25.571611,608_2050_AWR,608,AWR,2050,6.953076
24,-2.206302,999_2021_AWR,999,AWR,2021,1.319345
25,-0.463110,999_2025_AWR,999,AWR,2025,1.158689


In [34]:
# Step 1 - Extract 2050 harvest carbon numbers
df_hrv_2050 = df_hrv[df_hrv['Year'] == 2050]
# If you want to filter out ASV, uncomment the below
# df_hrv_2050 = df_hrv_2050[df_hrv_2050['MgmtID'] != 'ASV']

df_hrv_2050 = df_hrv_2050[['Year', 'StandID', 'MgmtID', 'Merch_Carbon_Stored']]
# As a cheat, we relabel the year to 2021 so it merges with 2021 years
df_hrv_2050 = df_hrv_2050.assign(Year = 2021)
df_hrv_2050 = df_hrv_2050.rename(columns={
    'Merch_Carbon_Stored': 'Carbon_Stored_2050'
})

df_hrv_2050

,Year,StandID,MgmtID,Carbon_Stored_2050
3,2021,505,ASV,1.162175
7,2021,505,IFM,0.803028
11,2021,505,PLSQ,0.000000
15,2021,505,THNB,0.418785
19,2021,608,AWR,6.953076
23,2021,608,PLSQ,0.000000
27,2021,999,AWR,0.810735
31,2021,999,CAR,0.810735
35,2021,999,NAR,0.810735
39,2021,999,PLSQ,0.000000


In [35]:
# Step 2 - Combine with actual df and add harvest to 2021 numbers
dup_hrv = copy.deepcopy(df_hrv)

dup_hrv = dup_hrv.merge(df_hrv_2050,
            how='left',
            on=['Year', 'StandID', 'MgmtID'],
            validate='many_to_one'
            )

dup_hrv = dup_hrv.fillna(0)
dup_hrv['Net_Carbon'] = dup_hrv['Net_Carbon'] + dup_hrv['Carbon_Stored_2050']

dup_hrv

,Net_Carbon,Variable,StandID,MgmtID,Year,Merch_Carbon_Stored,Carbon_Stored_2050
0,1.284222,505_2021_ASV,505,ASV,2021,0.000083,1.162175
1,4.206501,505_2025_ASV,505,ASV,2025,0.179192,0.000000
2,3.996429,505_2030_ASV,505,ASV,2030,0.372516,0.000000
3,31.035625,505_2050_ASV,505,ASV,2050,1.162175,0.000000
4,-1.660694,505_2021_IFM,505,IFM,2021,1.492325,0.803028
5,1.496395,505_2025_IFM,505,IFM,2025,1.235478,0.000000
6,3.317268,505_2030_IFM,505,IFM,2030,1.043617,0.000000
7,20.950588,505_2050_IFM,505,IFM,2050,0.803028,0.000000
8,0.000000,505_2021_PLSQ,505,PLSQ,2021,0.000000,0.000000
9,4.322262,505_2025_PLSQ,505,PLSQ,2025,0.000000,0.000000


In [ ]:
# Step 3 - Do ASV harvest carbon (NOT DONE, COMMENT LEFT FOR REFERENCE)
#
# This should be systemized, but because it's a small # entries,
# I did it manually.
#
#   	Net_Carbon 	Variable     	StandID 	MgmtID 	Year 	Merch_Carbon_Stored 	Carbon_Stored_2050
# 0 	0.122047 	505_2021_ASV 	505     	ASV 	2021 	0.000083 	0.000000
# 1 	4.206501 	505_2025_ASV 	505     	ASV 	2025 	0.179192 	0.000000
# 2 	3.996429 	505_2030_ASV 	505     	ASV 	2030 	0.372516 	0.000000
# 3 	31.035625 	505_2050_ASV 	505     	ASV 	2050 	1.162175 	0.000000
# 
# So, merch_carbon for 2021 = 
#


In [37]:
df_hrv = dup_hrv.drop(columns=[
    'Carbon_Stored_2050', 'Merch_Carbon_Stored'
])

df_hrv

,Net_Carbon,Variable,StandID,MgmtID,Year
0,1.284222,505_2021_ASV,505,ASV,2021
1,4.206501,505_2025_ASV,505,ASV,2025
2,3.996429,505_2030_ASV,505,ASV,2030
3,31.035625,505_2050_ASV,505,ASV,2050
4,-1.660694,505_2021_IFM,505,IFM,2021
5,1.496395,505_2025_IFM,505,IFM,2025
6,3.317268,505_2030_IFM,505,IFM,2030
7,20.950588,505_2050_IFM,505,IFM,2050
8,0.000000,505_2021_PLSQ,505,PLSQ,2021
9,4.322262,505_2025_PLSQ,505,PLSQ,2025


In [39]:
df_hrv.to_csv('./run4_correctcarboncosts/Objective_NetSeq_NoHarvest_08172022.csv',
             columns=['Variable', 'Net_Carbon'],
             index=False
             )

## 3 - Add Carbon Costs
Here, we take a user specified cost of each mgmt by forest type. 

We probably also need to scale it up by years since not all periods are equal length but that's future work -\\\_(ツ)_/-

In [9]:
COSTS_DICT = {
    '167N': {
        'PLSQ': 0,
        'THNB': 10
    },
    '167S': {
        'PLSQ': 0,
        'THNB': 10
    },
    '505': {
        'PLSQ': 0,
        'ASV': 0,
        'IFM': 0,
        'THNB': 10
    },
    '608': {
        'PLSQ': 0,
        'AWR': 0
    },
    '999': {
        'PLSQ': 0,
        'AWR': 0,
        'CAR': 0,
        'NAR': 0,
        'SAR': 0
    }
}

In [107]:
def create_dataframe_from_costs (cost_dict: dict) -> dict:
    # The format of the dict isn't actually all that good for converting to a dataframe
    # so we need some ~finangeling~ for it to work    
    pd_dict = {
        'StandID': [],
        'MgmtID': [],
        'Mgmt_Cost': []
    }

    for_types = cost_dict.keys()
    for f in for_types:
        mgmts = cost_dict[f].keys()
        for mg in mgmts:
            cost = cost_dict[f][mg]

            pd_dict['StandID'].append(f)
            pd_dict['MgmtID'].append(mg)
            pd_dict['Mgmt_Cost'].append(cost)
    
    _df = pd.DataFrame.from_dict(pd_dict)
    return _df

create_dataframe_from_costs(COSTS_DICT)

,StandID,MgmtID,Mgmt_Cost
0,167N,PLSQ,0
1,167N,THNB,10
2,167S,PLSQ,0
3,167S,THNB,10
4,505,PLSQ,0
5,505,ASV,0
6,505,IFM,0
7,505,THNB,10
8,608,PLSQ,0
9,608,AWR,0


In [109]:
# Now we append the management cost to the main df, and subtract it from net_carbon
def subtract_costs_df (_df_all, _cost_dict: dict):
    '''
    Returns a copy of the df_all, but where the 'Net_Carbon' column
    had the management costs in df_costs removed.
    '''
    _df_costs = create_dataframe_from_costs(_cost_dict)
    
    # The underscores in front of the variables is to avoid any scope
    # ex: df_mgcosts exists outside this function, so using _df_mgcosts removes
    #     the possibility of editing the outside-function df
    _df_mgcost = _df_all.merge(
        _df_costs,
        how='left',
        on=['StandID', 'MgmtID'],
        validate='many_to_one'
    )
    
    _df_mgcost['Net_Carbon'] = _df_mgcost['Net_Carbon'] - _df_mgcost['Mgmt_Cost']
    _df_mgcost = _df_mgcost.drop(columns=['Mgmt_Cost'])
    
    return _df_mgcost

df_mgcost = subtract_costs_df(df_hrv, COSTS_DICT)
df_mgcost

,Variable,StandID,MgmtID,Year,Net_Carbon
0,167N_2021_PLSQ,167N,PLSQ,2021,0.000000
1,167N_2025_PLSQ,167N,PLSQ,2025,1.848362
2,167N_2030_PLSQ,167N,PLSQ,2030,-0.386124
3,167N_2050_PLSQ,167N,PLSQ,2050,6.949192
4,167N_2021_THNB,167N,THNB,2021,-10.584286
5,167N_2025_THNB,167N,THNB,2025,-8.955025
6,167N_2030_THNB,167N,THNB,2030,-8.910110
7,167N_2050_THNB,167N,THNB,2050,11.187469
8,167S_2021_PLSQ,167S,PLSQ,2021,0.000000
9,167S_2025_PLSQ,167S,PLSQ,2025,5.697899


## Generating & Exporting Objective Files
In this section, we auto-generate 100s of objective files by varying different costs of managements.

To show another capabability, we afterwards fix the cost of a management and vary the harvest carbon,
again generating 100s of objective files.

Would be interesting also to see the regions (2D, 3D, 4D etc) grid where PLSQ is preffered to management
See: https://plotly.com/python/trisurf/

In [13]:
# df_mgcost.to_csv('./ObjectiveFile_MngCost_0802022.csv',
#               columns=['Variable', 'Net_Carbon'],
#               index=False
#              )

In [26]:
df_mgcost.sort_values(['Year', 'StandID', 'MgmtID'])

,Variable,StandID,MgmtID,Year,Net_Carbon
0,167N_2021_PLSQ,167N,PLSQ,2021,0.000000
4,167N_2021_THNB,167N,THNB,2021,-10.584286
8,167S_2021_PLSQ,167S,PLSQ,2021,0.000000
12,167S_2021_THNB,167S,THNB,2021,-10.931026
16,505_2021_ASV,505,ASV,2021,0.122047
20,505_2021_IFM,505,IFM,2021,-2.463722
24,505_2021_PLSQ,505,PLSQ,2021,0.000000
28,505_2021_THNB,505,THNB,2021,-11.386257
32,608_2021_AWR,608,AWR,2021,-18.667194
36,608_2021_PLSQ,608,PLSQ,2021,0.000000


Here's how I want to try sampling different costs:

- **167N** the cost of THNB to be inside the interval \[-1, 10\]
- **505** is tricky as there are 3 different managements, so we can take a cube where the cost for each type ranges from \[-1, 10\]. Alternatively we can vary each one individually.
- **608** has just one mng, AWR
- **999** again has a couple managements. We have AWR, CAR, NAR, SAR, all of which we can range from \[-1, 10\]

When a management is not the active one being managed, we set the cost of all others to 10 (except PLSQ which is always 0)

For each interval, I want to take at least 100 samples. This means overall we're making 400 objective files (999 has 4 different costs that need varying).

In [144]:
SAMPLES_PER_MNG = 100
COST_LOW = -1
COST_HIGH = 10
# The managements to vary the cost of
MNGS_DICT = {
    '167N': ['THNB'],
    '167S': ['THNB'],
    '505': ['ASV', 'IFM', 'THNB'],
    '608': ['AWR'],
    '999': ['AWR', 'CAR', 'NAR', 'SAR']
}

OBJ_EXPORT_FOLDER = '/home/velcro/Documents/Professional/NJDEP/TechWork/ForMOM/minimodel-running/run3_carboncosts/runs_varying_carboncost/objectives/'
COST_EXPORT_FOLDER = '/home/velcro/Documents/Professional/NJDEP/TechWork/ForMOM/minimodel-running/run3_carboncosts/runs_varying_carboncost/'


In [113]:
# Now comes the fun part of actually doing this
max_number_mngs = max([len(MNGS_DICT[ft]) for ft in FOR_TYPES])
total_num_samples = max_number_mngs * SAMPLES_PER_MNG

In [114]:
samples_per_ft_per_mng = {}

for ft in FOR_TYPES:
    num_mng = len(MNGS_DICT[ft])
    sample_per_mng = total_num_samples // num_mng
    # It's important we get exactly total_num_samples datapoints,
    # so the samples for the last mng might have a little extra
    sample_last_mng = total_num_samples - (num_mng - 1) * sample_per_mng
    
    ft_mngs = MNGS_DICT[ft]
    ft_num_mngs_dict = {}
    
    for ind, mng in enumerate(ft_mngs):
        if ind == num_mng - 1:
            ft_num_mngs_dict[mng] = sample_last_mng
        else:
            ft_num_mngs_dict[mng] = sample_per_mng
    
#     print(f"For ft {ft}, we have the following # samples per mng")
#     print(ft_num_mngs_dict)
#     print()
    samples_per_ft_per_mng[ft] = ft_num_mngs_dict

samples_per_ft_per_mng

{'167S': {'THNB': 400},
 '505': {'ASV': 133, 'IFM': 133, 'THNB': 134},
 '167N': {'THNB': 400},
 '999': {'AWR': 100, 'CAR': 100, 'NAR': 100, 'SAR': 100},
 '608': {'AWR': 400}}

In [125]:
# Generate dictionary of parallel arrays showing the costs in each run

def_cost = COST_HIGH
ind = 0

costs_across_runs = {}
for ft in FOR_TYPES:
    costs_across_runs[ft] = {}
    
    # PLSQ is always 0 cost
    costs_across_runs[ft]['PLSQ'] = list(np.linspace(0, 0, num=total_num_samples, endpoint=True))
    sum_inds = 0
    
    for mng in MNGS_DICT[ft]:
        costs_across_runs[ft][mng] = list(np.linspace(
            COST_HIGH, 
            COST_HIGH, 
            num=total_num_samples, 
            endpoint=True
        ))
        
        n_sample = samples_per_ft_per_mng[ft][mng]
        start_ind = sum_inds
        end_ind = start_ind + n_sample
        
        costs_across_runs[ft][mng][start_ind:end_ind] = list(np.linspace(
            COST_LOW,
            COST_HIGH,
            num=n_sample,
            endpoint=True
        ))
        
        # This needs to be at the end
        sum_inds = end_ind


costs_across_runs

{'167S': {'PLSQ': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,


In [147]:
# Now, build the objective files!
# This means, for each sample, we build a COST_DICT, make a dataframe from it, and export
all_costs = []

for ind in range(total_num_samples):
    baseline_costs = {}
    
    for ft in FOR_TYPES:
        baseline_costs[ft] = {}
        mngs = costs_across_runs[ft].keys()
        
        for mng in mngs:
            baseline_costs[ft][mng] = costs_across_runs[ft][mng][ind]
    all_costs.append(baseline_costs)
    
    df_objective = subtract_costs_df(df_hrv, baseline_costs)
    
    # Export this objective file
    filename = f'ObjectiveFile_VaryingCost_0802022_{ind}.csv'
    df_objective.to_csv(
        OBJ_EXPORT_FOLDER + filename,
        columns=['Variable', 'Net_Carbon'],
        index=False
    )    

In [146]:
# To make exporting the costs easier, a dataframe is constructed

# This is what the dataframe will look like:
#
# run  StandID  MgmtID  Mgmt_Cost
# 0    167N     PLSQ    0
# 0    167N     THNB    -1
# 0    167S     PLSQ    0
#  .     .       .      .
#  .     .       .      .
#  .     .       .      .
# 235  999      Nar    2.88
# 235  505      IFM    7.5
# 235  505      THNB   10.0
# 235  505      PLSQ   10.0
df_costs_all_runs = None

for ind, costs in enumerate(all_costs):
    df_indcost = create_dataframe_from_costs(costs)
    df_indcost.insert(loc=0, column='Run', value=ind)
    
    if type(df_costs_all_runs) == type(None):
        df_costs_all_runs = df_indcost
    else:
        df_costs_all_runs = pd.concat([df_costs_all_runs, df_indcost])

df_costs_all_runs.to_csv(COST_EXPORT_FOLDER + 'RunCosts.csv')
        
df_costs_all_runs

,Run,StandID,MgmtID,Mgmt_Cost
0,0,167S,PLSQ,0.0
1,0,167S,THNB,-1.0
2,0,505,PLSQ,0.0
3,0,505,ASV,-1.0
4,0,505,IFM,10.0
...,...,...,...,...
10,399,999,CAR,10.0
11,399,999,NAR,10.0
12,399,999,SAR,10.0
13,399,608,PLSQ,0.0


## Exporting Objective Files